#### 1. Calculate flight delays from the flight data collected.
##### Note: We are not using the site API as we are only considering flights between selected airports 

In [1]:
!pip3 install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# Function to calculated delay time in min
import pandas as pd

def calcDelayInMin(x):
    dep_delay, arr_delay = 0, 0
    
    if pd.notna(x['departure.revisedTime.utc']):
        diff = x['departure.revisedTime.utc'] - x['departure.scheduledTime.utc']
        if diff.total_seconds() > 0:
            dep_delay = diff.total_seconds() / 60
        
    if pd.notna(x['arrival.revisedTime.utc']):
        diff = x['arrival.revisedTime.utc'] - x['arrival.scheduledTime.utc']
        if diff.total_seconds() > 0:
            arr_delay = diff.total_seconds() / 60
            
    # If both departure and arrival delays are positive, set the smaller one to 0
    if (dep_delay >0 and arr_delay > 0):
        if dep_delay > arr_delay:
            arr_delay = 0
        else:
            dep_delay = 0
            
    return pd.Series([dep_delay, arr_delay], index=['depDelayTimeInMin', 'arrDelayTimeInMin'])

In [32]:
#Get flight data from csv file and format it

import pandas as pd
flightsData = pd.read_csv(r"C:\Guvi-Projects\Flight-Analytics\Data\finalFlightsData2.csv")

#Datetime coversion
date_cols = ['departure.scheduledTime.utc','departure.revisedTime.utc',
           'arrival.scheduledTime.utc','arrival.revisedTime.utc']
flightsData[date_cols] = flightsData[date_cols].apply(pd.to_datetime, errors='coerce')

#Select required columns
reqCols = ['number','status','airline.name','airline.iata','aircraft.reg','origin','destination',
           'departure.scheduledTime.utc','departure.scheduledTime.local', 'departure.revisedTime.utc',
           'departure.revisedTime.local','arrival.scheduledTime.utc','arrival.scheduledTime.local',
           'arrival.revisedTime.utc','arrival.revisedTime.local']

flightsData = flightsData[reqCols]

# Calculate delay
flightsData[['depDelayTimeInMin', 'arrDelayTimeInMin']] = flightsData.apply(calcDelayInMin, axis=1)

# Statistics for departure delays
dep_delay = flightsData.groupby(['origin', flightsData['departure.scheduledTime.utc'].dt.date], as_index=False)['depDelayTimeInMin'].agg(['sum', 'mean', 'count'])
dep_delay = dep_delay[dep_delay['sum']>0]
#dep_delay
dep_delay_cnt = flightsData[flightsData['depDelayTimeInMin']>0].groupby(['origin', flightsData['departure.scheduledTime.utc'].dt.date], as_index=False)['origin'].agg(['count'])

# Statistics for arrival delays
arr_delay = flightsData.groupby(['destination', flightsData['arrival.scheduledTime.utc'].dt.date], as_index=False)['arrDelayTimeInMin'].agg(['sum', 'mean', 'count'])
arr_delay = arr_delay[arr_delay['sum']>0]
arr_delay

arr_delay_cnt=flightsData[flightsData['arrDelayTimeInMin']>0].groupby(['destination', flightsData['arrival.scheduledTime.utc'].dt.date], as_index=False)['destination'].agg(['count'])

In [33]:
# Get cancelled flights data
flightsData['status'].unique()
cancelledDepFlights = flightsData[flightsData['status']=='Canceled'].groupby(['origin', flightsData['departure.scheduledTime.utc'].dt.date])['status'].agg(['count'])
cancelledDepFlights

cancelledArrFlights = flightsData[flightsData['status']=='Canceled'].groupby(['destination', flightsData['arrival.scheduledTime.utc'].dt.date])['status'].agg(['count'])
cancelledArrFlights

count
destination arrival.scheduledTime.utc       
BLR         2025-12-23                     4
            2025-12-24                     4
            2025-12-25                     1
            2025-12-26                     4
            2025-12-27                     1
BOM         2025-12-24                     2
CDG         2025-12-27                     3
DXB         2025-12-24                     1
JFK         2025-12-26                     3
            2025-12-27                    51
LHR         2025-12-23                     3
            2025-12-25                     2
            2025-12-27                     2
SFO         2025-12-23                     4
            2025-12-27                     5
VNS         2025-12-23                     2

In [34]:
#Merge cancelled flights with departure delay data
dep_delay = dep_delay.merge(cancelledDepFlights, left_on=['origin', 'departure.scheduledTime.utc'], right_on=['origin', 'departure.scheduledTime.utc'], how='left', suffixes=('', '_canceled')).fillna(0)
dep_delay = dep_delay.merge(dep_delay_cnt, left_on=['origin', 'departure.scheduledTime.utc'], right_on=['origin', 'departure.scheduledTime.utc'], how='left', suffixes=('', '_delayed')).fillna(0)
arr_delay = arr_delay.merge(cancelledArrFlights, left_on=['destination', 'arrival.scheduledTime.utc'], right_on=['destination', 'arrival.scheduledTime.utc'], how='left', suffixes=('', '_canceled')).fillna(0)
arr_delay = arr_delay.merge(arr_delay_cnt, left_on=['destination', 'arrival.scheduledTime.utc'], right_on=['destination', 'arrival.scheduledTime.utc'], how='left', suffixes=('', '_delayed')).fillna(0)

arr_delay.head()

destination arrival.scheduledTime.utc     sum       mean  count  \
0         AMD                2025-12-24    25.0   1.666667     15   
1         AMD                2025-12-26    43.0   1.592593     27   
2         AMD                2025-12-27     6.0   0.200000     30   
3         BLR                2025-12-23  3091.0  18.289941    169   
4         BLR                2025-12-24  1548.0  20.103896     77   

   count_canceled  count_delayed  
0             0.0              1  
1             0.0              3  
2             0.0              2  
3             4.0             77  
4             4.0             43

In [35]:
#calculate average delay

dep_delay['avg_delay'] = dep_delay['sum']/dep_delay['count_delayed']
dep_delay['isArrival'] = 0
dep_delay = dep_delay.drop('sum', axis=1)
dep_delay.rename(columns={'origin': 'airport','departure.scheduledTime.utc': 'travelDate'}, inplace=True)

arr_delay['avg_delay'] = arr_delay['sum']/arr_delay['count_delayed']
arr_delay['isArrival'] = 1
arr_delay = arr_delay.drop('sum', axis=1)
arr_delay.rename(columns={'destination': 'airport', 'arrival.scheduledTime.utc': 'travelDate'}, inplace=True)

#concat the data
delayData = pd.concat([dep_delay, arr_delay], ignore_index=True)
delayData

airport  travelDate       mean  count  count_canceled  count_delayed  \
0      AMD  2025-12-23   3.296875     64             0.0             12   
1      AMD  2025-12-24   6.902439     41             0.0             12   
2      AMD  2025-12-25   4.250000     20             0.0              7   
3      AMD  2025-12-26  10.655172     29             0.0              5   
4      AMD  2025-12-27   2.608696     23             0.0              4   
..     ...         ...        ...    ...             ...            ...   
76     SFO  2025-12-23  13.295082    122             4.0             18   
77     SFO  2025-12-24  21.692308    130             0.0             38   
78     SFO  2025-12-25   6.344262     61             0.0             11   
79     SFO  2025-12-26  24.224138    116             0.0             22   
80     SFO  2025-12-27  18.040541     74             5.0             10   

     avg_delay  isArrival  
0    17.583333          0  
1    23.583333          0  
2    12.142857          0  
3    61.800000          0  
4    15.000000          0  
..         ...        ...  
76   90.111111          1  
77   74.210526          1  
78   35.181818          1  
79  127.727273          1  
80  133.500000          1  

[81 rows x 8 columns]

#### 2. Store it in MySql database

In [38]:
import mysql.connector
import pandas as pd
connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "0123456789",
    database='FlightDB'
)
cursor = connection.cursor()

data = list(delayData.itertuples(index=False,name=None ))

arrDelayCreatequery = """CREATE TABLE IF NOT EXISTS FlightDelays(
            airport VARCHAR(10), 
            travelDate DATE, 
            medianDelayMin FLOAT, 
            totalFlights INT, 
            canceledFlights INT,
            delayedFlights INT, 
            avgDelayMin FLOAT,
            isArrival bool,
            foreign key(airport) references Airports(iata))"""
            
cursor.execute(arrDelayCreatequery)

arrDelayInsertQuery = r"INSERT INTO FlightDelays (airport, travelDate, medianDelayMin, totalFlights, canceledFlights, delayedFlights, avgDelayMin, isArrival) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s )"
cursor.executemany(arrDelayInsertQuery, data)
connection.commit()